In [1]:
import open3d as o3d
from open3d.web_visualizer import draw

import os
import open3d.ml as _ml3d
import open3d.ml.torch as ml3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


In [2]:
cube_red = o3d.geometry.TriangleMesh.create_box(1, 2, 4)
cube_red.compute_vertex_normals()
cube_red.paint_uniform_color((1.0, 0.0, 0.0))
draw(cube_red)

WebVisualizer(window_uid='window_0')

In [3]:
cube_blue = o3d.geometry.TriangleMesh.create_box(1, 2, 4)
cube_blue.compute_vertex_normals()
cube_blue.paint_uniform_color((0.0, 0.0, 1.0))
draw(cube_blue)

WebVisualizer(window_uid='window_1')

In [2]:
cfg_file = "ml3d/configs/pointpillars_kitti.yml"
cfg = _ml3d.utils.Config.load_from_file(cfg_file)

model = ml3d.models.PointPillars(**cfg.model)
cfg.dataset['dataset_path'] = "/data/cheng443/kitti/object/"
dataset = ml3d.datasets.KITTI(cfg.dataset.pop('dataset_path', None), **cfg.dataset)
pipeline = ml3d.pipelines.ObjectDetection(model, dataset=dataset, device="gpu", **cfg.pipeline)

In [3]:
# download the weights.
ckpt_folder = "./logs/"
os.makedirs(ckpt_folder, exist_ok=True)
ckpt_path = ckpt_folder + "pointpillars_kitti_202012221652utc.pth"
pointpillar_url = "https://storage.googleapis.com/open3d-releases/model-zoo/pointpillars_kitti_202012171738utc.pth"
if not os.path.exists(ckpt_path):
    cmd = "wget {} -O {}".format(pointpillar_url, ckpt_path)
    os.system(cmd)


In [4]:
# load the parameters.
pipeline.load_ckpt(ckpt_path=ckpt_path)

test_split = dataset.get_split("test")
data = test_split.get_data(0)


INFO - 2021-07-22 16:53:28,238 - object_detection - Loading checkpoint ./logs/pointpillars_kitti_202012221652utc.pth
INFO - 2021-07-22 16:53:28,307 - kitti - Found 7518 pointclouds for test


In [5]:
# run inference on a single example.
# returns dict with 'predict_labels' and 'predict_scores'.
result = pipeline.run_inference(data)

print(result)



[[box:1 (class=Pedestrian, conf=0.8616338), box:2 (class=Pedestrian, conf=0.36491564), box:3 (class=Pedestrian, conf=0.16166633), box:4 (class=Pedestrian, conf=0.13255985), box:5 (class=Pedestrian, conf=0.106997475), box:6 (class=Cyclist, conf=0.45997277), box:7 (class=Cyclist, conf=0.27199927), box:8 (class=Car, conf=0.6951123), box:9 (class=Car, conf=0.49514803), box:10 (class=Car, conf=0.19838047), box:11 (class=Car, conf=0.13000016), box:12 (class=Car, conf=0.11013488)]]


In [6]:
# evaluate performance on the test set; this will write logs to './logs'.
pipeline.run_test()

INFO - 2021-07-22 16:53:40,203 - object_detection - DEVICE : cuda
INFO - 2021-07-22 16:53:40,206 - object_detection - Logging in file : ./logs/PointPillars_KITTI_torch/log_test_2021-07-22_16:53:40.txt
INFO - 2021-07-22 16:53:40,210 - kitti - Found 7518 pointclouds for test
INFO - 2021-07-22 16:53:40,212 - object_detection - Initializing from scratch.
INFO - 2021-07-22 16:53:40,215 - object_detection - DEVICE : cuda
INFO - 2021-07-22 16:53:40,217 - object_detection - Logging in file : ./logs/PointPillars_KITTI_torch/log_valid_2021-07-22_16:53:40.txt
INFO - 2021-07-22 16:53:40,219 - kitti - Found 3769 pointclouds for validation
INFO - 2021-07-22 16:53:40,220 - object_detection - Started validation
validation: 100%|██████████| 3769/3769 [02:17<00:00, 27.50it/s]
/home/cheng443/.virtualenvs/open3D/lib/python3.6/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with di

TypeError: only size-1 arrays can be converted to Python scalars